In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# Load data
X = np.zeros([1,4])
y = np.zeros(1)
for i in range(1,18):
    data = np.load(r'D:\gradddddddddddddddddd\newdata\data_%i.npy' % i)
    for j in range(len(data)):
        if data[j,4] < 2500:
            X = np.append(X,np.array([np.hstack((data[j,2:5],data[j,6]))]),axis=0)
#            X = np.append(X,np.array([data[j,3:5]]),axis=0)
            y = np.append(y,data[j,5])
#X3 = X[:,0] - X[:,1]
#X = np.concatenate((X, np.array([X3]).T), axis=1)
X = np.delete(X, 0, 0)
y = np.delete(y, 0)
X[:,0] = X[:,0]**(1./3.)

y[y<10] = 0
y[y>=10] = 1
#np.save(r'D:\gradddddddddddddddddd\newdata\X_ATM.npy', X)
#np.save(r'D:\gradddddddddddddddddd\newdata\y_ATM.npy', y)

In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [4]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)] # features feed

In [15]:
# build model
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units = [4,4,4],
                                            n_classes = 2,
                                            optimizer = tf.train.AdamOptimizer,
                                            dropout=0.1,
                                            model_dir="/tmp/atm_2")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001710600C080>, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_model_dir': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_environment': 'local', '_task_type': None, '_master': '', '_num_worker_replicas': 0, '_is_chief': True}


In [16]:
def get_train_inputs():
    
    x = tf.constant(X_train)
    y = tf.constant(y_train)
    
    
    return x, y

In [17]:
print(classifier.fit(input_fn=get_train_inputs, steps=2000))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/atm_2\model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/atm_2\model.ckpt.
INFO:tensorflow:step = 2001, loss = 0.0286655
INFO:tensorflow:global_step/sec: 21.9169
INFO:tensorflow:step = 2101, loss = 0.0279479 (4.564 sec)
INFO:tensorflow:global_step/sec: 22.1315
INFO:tensorflow:step = 2201, loss = 0.0280392 (4.518 sec)
INFO:tensorflow:global_step/sec: 22.2284
INFO:tensorflow:step = 2301, loss = 0.0284357 (4.499 sec)
INFO:tensorflow:global_step/sec: 22.2218
INFO:tensorflow:step = 2401, loss = 0.0274516 (4.502 sec)
INFO:tensorflow:global_step/sec: 22.2272
INF

In [9]:
def get_test_inputs():
    x = tf.constant(X_test)
    y = tf.constant(y_test)

    return x, y

In [14]:
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                     steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-31-14:58:07
INFO:tensorflow:Restoring parameters from /tmp/atm_2\model.ckpt-2000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-07-31-14:58:08
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.988033, accuracy/baseline_label_mean = 0.0119675, accuracy/threshold_0.500000_mean = 0.988033, auc = 0.991727, global_step = 2000, labels/actual_label_mean = 0.0119675, labels/prediction_mean = 0.00541409, loss = 0.0222719, precision/positive_threshold_0.500000_mean = 0.0, recall/positive_threshold_0.500000_mean = 0.0

Test Accuracy: 0.988033

